In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
    from rank_bm25 import BM25Okapi
from langchain_core.documents import Document
from typing import List

# 1. 문서 로드 (예: 미리 크롤링해서 로컬에 저장된 텍스트 리스트)
raw_texts = [
    "첫 번째 문서 내용 ...",
    "두 번째 문서 내용 ...",
    # ... 다수의 문서
]

# 2. BM25 색인 구성
tokenized_corpus = [doc.split() for doc in raw_texts]
bm25 = BM25Okapi(tokenized_corpus)

# 3. FAISS 벡터스토어 준비 (이미 임베딩된 문서)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [Document(page_content=text) for text in raw_texts]
faiss_store = FAISS.from_documents(docs, embedding=embeddings)

# 4. GPT 모델 초기화
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

def hybrid_retrieve_and_generate(question: str) -> str:
    # 1차 필터: BM25로 상위 20개 후보 선별
    q_tokens = question.split()
    top_n = bm25.get_top_n(q_tokens, raw_texts, n=20)
    
    # 2차 필터: FAISS에서 BM25 결과만 다시 검색
    # (FAISS 검색 시, Candidate ID 리스트를 인자로 전달)
    candidate_docs = [Document(page_content=text) for text in top_n]
    sub_faiss = FAISS.from_documents(candidate_docs, embedding=embeddings)
    retrieved = sub_faiss.similarity_search(question, k=5)
    
    # 문맥 문자열 생성
    context = "\n---\n".join([doc.page_content for doc in retrieved])
    
    # RAG 프롬프트 구성
    prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following context to answer concisely (max 3 sentences).\n\n"
        f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    )
    
    # GPT에 전달해 답변 생성
    response = llm.invoke(prompt)
    return response.content

# 예시 실행
question = "LangGraph RAG 와 하이브리드 검색의 장점은 무엇인가?"
answer = hybrid_retrieve_and_generate(question)
print(answer)
